In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
df_ratings = pd.read_csv('ratings.csv')
df_movies = pd.read_csv('movies.csv')
df = pd.merge(df_ratings, df_movies, on='movieId')
del df['timestamp']
del df['genres']
df['rating'].value_counts()

4.0    26818
3.0    20047
5.0    13211
3.5    13136
4.5     8551
2.0     7551
2.5     5550
1.0     2811
1.5     1791
0.5     1370
Name: rating, dtype: int64

In [4]:
MAX_USER_ID = 610
MIN_USER_ID = 1
movie_names = df['title'].unique()
movie_names = movie_names.tolist()
movie_to_vector = {}
for movie in tqdm_notebook(movie_names):
    movie_to_vector[movie] = np.zeros((MAX_USER_ID,))
    for r in df[df['title'] == movie].iterrows():
        movie_to_vector[movie][r[1]['userId'] - MIN_USER_ID] = r[1]['rating']

In [3]:
movie_to_vector['Toy Story (1995)']

array([4. , 0. , 0. , 0. , 4. , 0. , 4.5, 0. , 0. , 0. , 0. , 0. , 0. ,
       0. , 2.5, 0. , 4.5, 3.5, 4. , 0. , 3.5, 0. , 0. , 0. , 0. , 0. ,
       3. , 0. , 0. , 0. , 5. , 3. , 3. , 0. , 0. , 0. , 0. , 0. , 0. ,
       5. , 0. , 0. , 5. , 3. , 4. , 5. , 0. , 0. , 0. , 3. , 0. , 0. ,
       0. , 3. , 0. , 0. , 5. , 0. , 0. , 0. , 0. , 0. , 5. , 4. , 0. ,
       4. , 0. , 2.5, 0. , 0. , 5. , 0. , 4.5, 0. , 0. , 0.5, 0. , 4. ,
       0. , 0. , 0. , 2.5, 0. , 0. , 0. , 4. , 0. , 0. , 3. , 3. , 4. ,
       0. , 3. , 0. , 0. , 5. , 0. , 4.5, 0. , 0. , 0. , 0. , 4. , 0. ,
       0. , 0. , 4. , 0. , 0. , 0. , 0. , 3. , 0. , 0. , 0. , 0. , 0. ,
       0. , 3.5, 0. , 4. , 0. , 0. , 4. , 0. , 0. , 0. , 0. , 0. , 3. ,
       0. , 2. , 0. , 3. , 4. , 0. , 4. , 0. , 0. , 3. , 4. , 0. , 0. ,
       3.5, 5. , 0. , 0. , 0. , 0. , 0. , 5. , 0. , 2. , 0. , 3. , 4. ,
       0. , 0. , 4.5, 4. , 4. , 0. , 0. , 0. , 0. , 5. , 3.5, 0. , 4.5,
       0. , 5. , 0. , 0. , 0. , 0. , 0. , 5. , 4. , 4. , 0. , 0.

## User 2 Item

In [50]:
from surprise import Dataset, Reader, KNNBasic
from surprise import SVD
from surprise import KNNWithMeans, KNNWithZScore, SlopeOne, SlopeOne, CoClustering, KNNBaseline
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.model_selection import cross_validate

In [8]:
df_for_surpise = df_ratings[['userId', 'movieId', 'rating']]
df_for_surpise.columns = ['uid', 'iid', 'rating']
df_for_surpise.head()

,uid,iid,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [9]:
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(df_for_surpise, reader)
algo = SVD ()

In [10]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8811  0.8753  0.8644  0.8766  0.8760  0.8747  0.0055  
MAE (testset)     0.6754  0.6700  0.6648  0.6741  0.6743  0.6717  0.0039  
Fit time          5.29    5.63    5.40    5.46    5.48    5.45    0.11    
Test time         0.19    0.15    0.15    0.14    0.18    0.16    0.02    


{'test_rmse': array([0.88109674, 0.87527054, 0.86443727, 0.87661081, 0.8759852 ]),
 'test_mae': array([0.67543603, 0.6699572 , 0.66481677, 0.67414952, 0.67429956]),
 'fit_time': (5.294065952301025,
  5.62656569480896,
  5.401913404464722,
  5.45524001121521,
  5.475681781768799),
 'test_time': (0.1914501190185547,
  0.14757061004638672,
  0.1505568027496338,
  0.14161944389343262,
  0.18354296684265137)}

In [15]:
trainset, testset = train_test_split(dataset, test_size=0.2)
algo.fit(trainset)
predictions = algo.test(testset)

In [53]:
df[df['userId']==algo.trainset.to_raw_uid(1)]

,userId,movieId,rating,title
168,483,1,4.0,Toy Story (1995)
537,483,47,3.5,Seven (a.k.a. Se7en) (1995)
740,483,50,4.5,"Usual Suspects, The (1995)"
1041,483,110,4.0,Braveheart (1995)
1345,483,223,3.5,Clerks (1994)
...,...,...,...,...
99540,483,105325,1.0,Bad Milo (Bad Milo!) (2013)
99541,483,109941,3.0,Puss in Boots: The Three Diablos (2012)
99542,483,109971,4.5,Ocho apellidos vascos (2014)
99543,483,112316,2.5,Jersey Boys (2014)


In [51]:
algo = KNNBaseline(k=40, sim_options={'name': 'cosine', 'user_based': True})

In [52]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8762  0.8762  0.8799  0.8798  0.8768  0.8778  0.0017  
MAE (testset)     0.6684  0.6702  0.6715  0.6747  0.6723  0.6715  0.0021  
Fit time          0.96    0.99    0.98    0.99    1.04    0.99    0.03    
Test time         2.32    2.59    2.40    2.30    2.36    2.40    0.11    


{'test_rmse': array([0.87623057, 0.87624448, 0.87986743, 0.87982181, 0.87684904]),
 'test_mae': array([0.66842281, 0.67023165, 0.67154532, 0.67472152, 0.67233194]),
 'fit_time': (0.9634175300598145,
  0.9851865768432617,
  0.9823710918426514,
  0.9942986965179443,
  1.0430529117584229),
 'test_time': (2.3243510723114014,
  2.5945684909820557,
  2.397670269012451,
  2.298935890197754,
  2.3610284328460693)}

In [55]:
trainset, testset = train_test_split(dataset, test_size=0.2)
algo.fit(trainset)
predictions = algo.test(testset)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [56]:
df[df['userId']==algo.trainset.to_raw_uid(1)]

,userId,movieId,rating,title
242,294,3,1.0,Grumpier Old Men (1995)
312,294,6,3.0,Heat (1995)
467,294,47,2.0,Seven (a.k.a. Se7en) (1995)
667,294,50,3.0,"Usual Suspects, The (1995)"
795,294,70,4.0,From Dusk Till Dawn (1996)
...,...,...,...,...
96190,294,3043,1.0,Meatballs 4 (1992)
96191,294,3412,2.0,"Bear, The (Ours, L') (1988)"
96197,294,3544,3.0,Shakes the Clown (1992)
96200,294,3667,1.0,Rent-A-Cop (1988)


## Item-based рекомендация

In [59]:
algo = KNNWithMeans(k=60, sim_options={'name': 'pearson_baseline', 'user_based': False})

In [60]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8836  0.8921  0.8819  0.8732  0.8812  0.8824  0.0060  
MAE (testset)     0.6730  0.6748  0.6726  0.6638  0.6709  0.6710  0.0038  
Fit time          14.17   14.11   14.11   14.16   14.39   14.19   0.10    
Test time         10.95   10.74   11.25   10.82   11

{'test_rmse': array([0.88360192, 0.8921337 , 0.88191253, 0.87323561, 0.88117732]),
 'test_mae': array([0.67297077, 0.67478866, 0.67261722, 0.6638179 , 0.67090671]),
 'fit_time': (14.172147035598755,
  14.10655951499939,
  14.111612558364868,
  14.158588647842407,
  14.38679552078247),
 'test_time': (10.949096441268921,
  10.742808103561401,
  11.249415159225464,
  10.815296411514282,
  11.1955406665802)}

In [61]:
trainset, testset = train_test_split(dataset, test_size=0.2)
algo.fit(trainset)
predictions = algo.test(testset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [63]:
df[df['userId']==algo.trainset.to_raw_uid(1)]

,userId,movieId,rating,title
760,569,50,3.0,"Usual Suspects, The (1995)"
1480,569,231,3.0,Dumb & Dumber (Dumb and Dumber) (1994)
2102,569,296,5.0,Pulp Fiction (1994)
2250,569,316,4.0,Stargate (1994)
2415,569,349,4.0,Clear and Present Danger (1994)
2727,569,356,3.0,Forrest Gump (1994)
3405,569,480,4.0,Jurassic Park (1993)
4105,569,590,4.0,Dances with Wolves (1990)
4287,569,592,3.0,Batman (1989)
20105,569,588,4.0,Aladdin (1992)


In [66]:
algo = KNNBaseline(k=60, sim_options={'name': 'pearson_baseline', 'user_based': False})

In [67]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8557  0.8472  0.8475  0.8572  0.8556  0.8526  0.0044  
MAE (testset)     0.6534  0.6455  0.6484  0.6536  0.6522  0.6506  0.0032  
Fit time          14.20   13.80   14.01   14.05   14.17   14.05   0.14    
Test time         11.67   11.73   12.06   11.98   11.

{'test_rmse': array([0.85571715, 0.8471665 , 0.84754157, 0.85720462, 0.85560958]),
 'test_mae': array([0.65339881, 0.64547976, 0.6483898 , 0.65360409, 0.6522082 ]),
 'fit_time': (14.200519561767578,
  13.801073551177979,
  14.013052225112915,
  14.050150871276855,
  14.173919200897217),
 'test_time': (11.67282485961914,
  11.72562289237976,
  12.06279706954956,
  11.975095987319946,
  11.747020244598389)}

In [68]:
trainset, testset = train_test_split(dataset, test_size=0.2)
algo.fit(trainset)
predictions = algo.test(testset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [69]:
df[df['userId']==algo.trainset.to_raw_uid(1)]

,userId,movieId,rating,title
140,391,1,3.0,Toy Story (1995)
334,391,6,4.0,Heat (1995)
504,391,47,3.0,Seven (a.k.a. Se7en) (1995)
712,391,50,4.0,"Usual Suspects, The (1995)"
810,391,70,1.0,From Dusk Till Dawn (1996)
...,...,...,...,...
97872,391,4965,3.0,"Business of Strangers, The (2001)"
97874,391,5470,4.0,The Importance of Being Earnest (1952)
97877,391,5477,5.0,Sex and Lucia (Lucía y el sexo) (2001)
97881,391,5537,5.0,Satin Rouge (2002)
